In [1]:
!pip install -q datasets
!pip install -q chromadb

In [12]:
! pip install pandas


In [11]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')

Generating train split: 100%|██████████| 16407/16407 [00:00<00:00, 76132.00 examples/s]


In [13]:
import pandas as pd 
df = data.to_pandas()
df.head()

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [14]:
len(df)

16407

In [33]:
df.Question[0]

'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'

In [21]:
df = df[:100]

In [22]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df,page_content_column="Answer")
df_data = loader.load()
df_data[0]

Document(metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}, page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.')

In [23]:
len(df_data)

100

In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings()

/tmp/ipykernel_1217018/950934551.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings()
/tmp/ipykernel_1217018/950934551.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [28]:
from langchain.vectorstores import Chroma

chromaDB = Chroma.from_documents(df_data,embedding=embedding_model)

In [30]:
from langchain_groq import ChatGroq
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chromaDB.as_retriever()
)

/tmp/ipykernel_1217018/2569822188.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(


In [36]:
qa.invoke('Who is at risk for Lymphocytic Choriomeningitis (LCM)? ')

{'query': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ',
 'result': 'Individuals at risk for Lymphocytic Choriomeningitis (LCM) include:\n\n1. Those who come into contact with urine, feces, saliva, or blood of wild mice.\n2. Owners of pet mice or hamsters that originate from contaminated colonies or are infected by wild mice.\n3. Laboratory workers who work with the virus or handle infected animals (although this risk can be minimized with proper precautions).\n4. Human fetuses who can acquire the infection vertically from an infected mother.\n5. Individuals who handle fresh urine, droppings, saliva, or nesting materials from infected rodents, or have these materials introduced into broken skin, the nose, eyes, or mouth.\n6. Recipients of organ transplants from an infected donor (rarely).\n\nIn general, anyone who comes into contact with infected rodents or their bodily fluids is potentially at risk for infection.'}

In [45]:
qa.invoke('what is 1235*1232')

{'query': 'what is 1235*1232',
 'result': "I don't know the answer to that question as it is not related to the provided context. However, I can calculate it for you: \n\n1235 * 1232 = 1,520,720."}

In [47]:
qa.invoke('who is president of America in 1990?')

{'query': 'who is president of America in 1990?',
 'result': 'The President of the United States in 1990 was George H.W. Bush.'}

In [37]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            """use this tool when answering medical knowledge queries to get
            more information about the topic"""
        )
    )
]

In [39]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm=llm,tools=tools,prompt=prompt)

/home/eps/Documents/Projects/LangChain-sandbox/venv/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

In [41]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The area of a square is calculated by multiplying the length of one side by itself. In this case, the length of one side is 2. So, the area of the square is 2 * 2 = 4.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square is calculated by multiplying the length of one side by itself. In this case, the length of one side is 2. So, the area of the square is 2 * 2 = 4.'}

In [42]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism diagnosisDiagnosing botulism can be challenging as the symptoms are similar to those of other diseases, such as Guillain-Barré syndrome, stroke, and myasthenia gravis. 

To diagnose botulism, a physician may consider the patient's history and physical examination, but these clues are usually not enough to confirm the diagnosis. Special tests may be needed to exclude other conditions and confirm botulism. These tests may include:

1. Brain scan
2. Spinal fluid examination
3. Nerve conduction test (electromyography, or EMG)
4. Tensilon test for myasthenia gravis
5. Tests for botulinum toxin and for bacteria that cause botulism, which can be performed at some state health department laboratories and at CDC.

The classic symptoms of botulism, such as double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness, can

{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square is calculated by multiplying the length of one side by itself. In this case, the length of one side is 2. So, the area of the square is 2 * 2 = 4.', additional_kwargs={}, response_metadata={})],
 'output': "To confirm the diagnosis of botulism in your patient, consider their history and physical examination, but also utilize special tests such as a brain scan, spinal fluid examination, nerve conduction test, and tests for botulinum toxin and the bacteria that cause botulism. Classic symptoms like double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness can also guide the diagnosis. In infants, look for lethargy, poor feeding, constipation, and a weak cry. A definitive diagnosis 

In [43]:
agent_executor.invoke({"input": """I dont understand can you explain simpler"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism diagnosisDiagnosing botulism can be challenging as the symptoms are similar to those of other diseases such as Guillain-Barré syndrome, stroke, and myasthenia gravis. 

To diagnose botulism, physicians consider the patient's history and physical examination, but special tests are often needed to confirm the diagnosis. These tests may include:

1. Brain scan
2. Spinal fluid examination
3. Nerve conduction test (electromyography, or EMG)
4. Tensilon test for myasthenia gravis
5. Tests for botulinum toxin and for bacteria that cause botulism, which can be performed at some state health department laboratories and at the CDC.

The classic symptoms of botulism, such as double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness, can also help guide the diagnosis. In infants, symptoms may include lethargy, poor feed

{'input': 'I dont understand can you explain simpler',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square is calculated by multiplying the length of one side by itself. In this case, the length of one side is 2. So, the area of the square is 2 * 2 = 4.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnosis?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To confirm the diagnosis of botulism in your patient, consider their history and physical examination, but also utilize special tests such as a brain scan, spinal fluid examination, nerve conduction test, and tests for botulinum toxin and the bacteria that cause botulism. Classic symptoms like double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness can als

In [50]:
agent_executor.invoke({"input": """Who is the president of America now?"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The President of the United States is Joe Biden. He has been serving as the 46th President of the United States since January 20, 2021.

> Finished chain.


{'input': 'Who is the president of America now?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square is calculated by multiplying the length of one side by itself. In this case, the length of one side is 2. So, the area of the square is 2 * 2 = 4.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnosis?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To confirm the diagnosis of botulism in your patient, consider their history and physical examination, but also utilize special tests such as a brain scan, spinal fluid examination, nerve conduction test, and tests for botulinum toxin and the bacteria that cause botulism. Classic symptoms like double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness can also gui